# models

> Review step state and working document model for Phase 3: Review & Commit

In [ ]:
#| default_exp models

In [ ]:
#| export
from typing import Optional, List, Dict, Any
from typing_extensions import TypedDict
from dataclasses import dataclass, field

from cjm_fasthtml_card_stack.core.models import CardStackUrls
from cjm_source_provider.models import SourceBlock

# Cross-package imports: primary owners are in page-specific packages
from cjm_transcript_segmentation.models import TextSegment
from cjm_transcript_vad_align.models import VADChunk

## ReviewStepState

TypedDict for Phase 3: Review & Commit step state.

In [ ]:
#| export
class ReviewStepState(TypedDict, total=False):
    """State for Phase 3: Review & Commit."""
    
    document_title: str  # Final document title
    validation_errors: List[str]  # List of validation issues
    is_validated: bool  # Whether document passed validation
    
    # Card stack view state
    focused_index: int  # Currently focused segment index
    visible_count: int  # Number of visible cards in viewport
    is_auto_mode: bool  # Whether card count is in auto-adjust mode
    card_width: int  # Card stack width in rem units
    
    # Audio playback state
    playback_speed: float  # Current playback speed (default 1.0)
    auto_navigate: bool  # Whether to auto-navigate on audio completion

## ReviewUrls

URL bundle for Phase 3 review route handlers and renderers.

In [ ]:
#| export
@dataclass
class ReviewUrls:
    """URL bundle for Phase 3 review route handlers and renderers."""
    
    # Card stack navigation and viewport (from cjm-fasthtml-card-stack library)
    card_stack: CardStackUrls = field(default_factory=CardStackUrls)
    
    # Audio controls (Phase 4)
    audio_src: str = ""  # Audio source route
    speed_change: str = ""  # Playback speed change handler
    toggle_auto_nav: str = ""  # Auto-navigate toggle handler
    replay_current: str = ""  # Replay current segment handler
    
    # Document metadata
    update_title: str = ""  # Document title update handler
    
    # Commit (Phase 5)
    commit: str = ""  # Commit handler route

## WorkingDocument

Container for all working data during the decomposition workflow.

In [ ]:
#| export
@dataclass
class WorkingDocument:
    """Container for workflow state during structure decomposition."""
    
    title: str = ""  # Document title
    media_type: str = "audio"  # Source media type ('audio', 'video', 'text')
    media_path: Optional[str] = None  # Path to primary source media
    
    # Source blocks (Phase 1 output)
    source_blocks: List[SourceBlock] = field(default_factory=list)  # Ordered source blocks
    
    # Combined raw text (for decomposition)
    combined_text: str = ""  # Concatenated text from all sources
    
    # Working segments (Phase 2 output, Phase 3 input/output)
    segments: List[TextSegment] = field(default_factory=list)  # Decomposed segments
    
    # VAD data (Phase 3)
    vad_chunks: List[VADChunk] = field(default_factory=list)  # VAD time ranges
    audio_duration: Optional[float] = None  # Total audio duration in seconds
    
    def to_dict(self) -> Dict[str, Any]:  # Dictionary representation
        """Convert to dictionary for JSON serialization."""
        return {
            'title': self.title,
            'media_type': self.media_type,
            'media_path': self.media_path,
            'source_blocks': [b.to_dict() for b in self.source_blocks],
            'combined_text': self.combined_text,
            'segments': [s.to_dict() for s in self.segments],
            'vad_chunks': [c.to_dict() for c in self.vad_chunks],
            'audio_duration': self.audio_duration
        }
    
    @classmethod
    def from_dict(
        cls,
        data: Dict[str, Any]  # Dictionary representation
    ) -> "WorkingDocument":  # Reconstructed WorkingDocument
        """Create from dictionary."""
        return cls(
            title=data.get('title', ''),
            media_type=data.get('media_type', 'audio'),
            media_path=data.get('media_path'),
            source_blocks=[SourceBlock(**b) for b in data.get('source_blocks', [])],
            combined_text=data.get('combined_text', ''),
            segments=[TextSegment.from_dict(s) for s in data.get('segments', [])],
            vad_chunks=[VADChunk.from_dict(c) for c in data.get('vad_chunks', [])],
            audio_duration=data.get('audio_duration')
        )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()